In [2]:
proj_id = 'cmat-315920'
root_path = '/home/jupyter'
code_path = root_path + '/MathVGerrmandering_CMAT_2021/src/'
for f in ['Core', 'Gerry', 'Variable', 'Crosswalks', 'Assignments', 'Shapes',
          'Census', 'Elections', 'Votes', 'Combined', 'Edges', 'Nodes', 'Graph', 'Districts']:
    %run -i {code_path+f+'.py'}

refresh_all = (
#     'crosswalks',
#     'assignments',
#     'shapes',
#     'census',
#     'elections',
#     'votes_all',
#     'votes_hl',
#     'combined',
#     'edges',
#     'nodes',
#     'graph',
)

refresh_tbl = (
#     'crosswalks',
#     'assignments',
#     'shapes',
#     'census',
#     'elections',
#     'votes_all',
#     'votes_hl',
#     'combined',
#     'edges',
#     'nodes',
#     'graph',
)

self = Gerry(abbr = 'TX',
             level = 'cntyvtd',
             district_type='cd',
             simplification=0,
             num_colors=10,
             election_filters=(
                 "office='President' and race='general'",
                 "office='USSen' and race='general'",
#                  "office like 'USRep%' and race='general'",
             ),
             refresh_tbl=refresh_tbl, refresh_all=refresh_all,
             num_steps=10000,
            )

start = time.time()

idx = self.nodes.df.nlargest(2, 'total_pop').index
self.nodes.df.loc[idx[0], 'cd'] = '37'
self.nodes.df.loc[idx[1], 'cd'] = '38'

self.MCMC()
# self.agg_plans(agg_polygon_steps=False)
#                (0,10))#True)#agg_polygon_steps=list(range(3,15)))
# self.stack_plans()

elapsed = time.time() - start
h, m = divmod(elapsed, 3600)
m, s = divmod(m, 60)
print(f'{int(h)}hrs {int(m)}min {s:.2f}sec elapsed')

user_name (default=cook) 
random_seed (default=1) 


Get crosswalks_TX_2020_tabblock   ... not necessary ... success
Get assignments_TX_2020_tabblock  ... tabblock table exists ... success
Get shapes_TX_2020_tabblock       ... tabblock table exists ... success
Get census_TX_2020_tabblock       ... tabblock table exists ... success
Get elections_TX_2020_tabblock    ... tabblock table exists ... success
Get votes_all_TX_2020_tabblock    ... tabblock table exists ... success
Get votes_hl_TX_2020_tabblock     ... tabblock table exists ... success
Get combined_TX_2020_cntyvtd_cd   ... cntyvtd table exists ... success
Get nodes_TX_2020_cntyvtd         ... success
Get edges_TX_2020_cntyvtd         ... cntyvtd table exists ... success
Get graph_TX_2020_cntyvtd_cd      ... gpickle exists ... success
Get cd TX 2020 cntyvtd cd         ... success
MCMC plan_1 ... pushing ... current pop_imbalance=88.18 ... setting tol=88.19% ... recombed cd 08 and 10 with pop_imbalance=84.42% ... -4801659205531664948 ... success

MCMC plan_2 ... pushing ... current 

In [3]:
%pwd

'/home/jupyter/MathVGerrmandering_CMAT_2021'

In [1]:
from src import *
# from src.Core import *
# from src.Assignments import Assignments
# pd.Series(np.arange(10))

user_name (default=cook) 
random_seed (default=1) 


getting states


In [ ]:
proj_id = 'cmat-315920'
root_path = '/home/jupyter'
code_path = root_path + '/MathVGerrmandering_CMAT_2021/src/'
for f in ['Core', 'Gerry', 'Variable', 'Crosswalks', 'Assignments', 'Shapes',
          'Census', 'Elections', 'Votes', 'Combined', 'Edges', 'Nodes', 'Graph', 'Districts']:
    %run -i {code_path+f+'.py'}

refresh_all = (
#     'crosswalks',
#     'assignments',
#     'shapes',
#     'census',
#     'elections',
#     'votes_all',
#     'votes_hl',
#     'combined',
#     'edges',
#     'nodes',
#     'graph',
)

refresh_tbl = (
#     'crosswalks',
#     'assignments',
#     'shapes',
#     'census',
#     'elections',
#     'votes_all',
#     'votes_hl',
#     'combined',
#     'edges',
#     'nodes',
#     'graph',
)

self = Gerry(abbr = 'TX',
             level = 'cntyvtd',
             district_type='cd',
             simplification=0,
             num_colors=10,
             election_filters=(
                 "office='President' and race='general'",
                 "office='USSen' and race='general'",
#                  "office like 'USRep%' and race='general'",
             ),
             refresh_tbl=refresh_tbl, refresh_all=refresh_all,
             num_steps=10000,
            )

start = time.time()

idx = self.nodes.df.nlargest(2, 'total_pop').index
self.nodes.df.loc[idx[0], 'cd'] = '37'
self.nodes.df.loc[idx[1], 'cd'] = '38'

self.MCMC()
# self.agg_plans(agg_polygon_steps=False)
#                (0,10))#True)#agg_polygon_steps=list(range(3,15)))
# self.stack_plans()

elapsed = time.time() - start
h, m = divmod(elapsed, 3600)
m, s = divmod(m, 60)
print(f'{int(h)}hrs {int(m)}min {s:.2f}sec elapsed')

In [3]:
grp = self.nodes.df.groupby(self.district_type)
self.dict  = {k:tuple(sorted(v)) for k,v in grp.groups.items()}
self.tuple = tuple(sorted(self.dict.values()))
self.tuple[-1]

('48491',)

In [4]:
hash(tuple(sorted(self.graph.graph.edges)))

-7985129636994162723

In [5]:
w = [tuple(sorted(v)) for k,v in self.districts.dict.items()]
q = tuple(sorted(tuple(sorted(v)) for k,v in self.districts.dict.items()))
type(q), q

(tuple,
 (('021001001',
   '021001002',
   '021001004',
   '021002005',
   '021002006',
   '021002009',
   '021002010',
   '021002011',
   '021004017',
   '021004019',
   '021004020',
   '021004021',
   '055000100',
   '055000101',
   '055000103',
   '055000105',
   '055000107',
   '055000108',
   '055000109',
   '055000114',
   '055000115',
   '055000116',
   '055000118',
   '055000302',
   '055000303',
   '055000400',
   '055000401',
   '055000402',
   '055000407',
   '055000408',
   '055000409',
   '055000411',
   '055000412',
   '209000120',
   '209000125',
   '209000126',
   '209000127',
   '209000129',
   '209000130',
   '209000229',
   '209000231',
   '209000233',
   '209000234',
   '287000101',
   '287000102',
   '287000103',
   '287000201',
   '287000202',
   '287000203',
   '287000304',
   '287000401',
   '287000402',
   '287000403',
   '287000404',
   '453000101',
   '453000103',
   '453000104',
   '453000105',
   '453000106',
   '453000108',
   '453000114',
   '453000115',


In [6]:
nx.is_connected

<function networkx.algorithms.components.connected.is_connected(G)>

In [8]:
user_name = 'acook'
level = 'cntyvtd'
# level = 'tract'
district_type = 'cd'
proj_id = 'cmat-315920'

root_path = f'/home/jupyter'

import os, pathlib, google, numpy as np, pandas as pd, geopandas as gpd, matplotlib.pyplot as plt, plotly.express as px
from google.cloud import aiplatform, bigquery
try:
    from google.cloud.bigquery_storage import BigQueryReadClient
except:
    os.system('pip install --upgrade google-cloud-bigquery-storage')
    from google.cloud.bigquery_storage import BigQueryReadClient

cred, proj = google.auth.default(scopes=["https://www.googleapis.com/auth/cloud-platform"])
bqclient   = bigquery.Client(credentials=cred, project=proj)
root_path  = pathlib.Path(root_path)
image_path = root_path / f'images/{district_type}/{level}'

tbl = f'{proj_id}.redistricting_data.{user_name}_plans_TX_2020_{level}_{district_type}_stacked'
df = (bqclient.query(f'select * from {tbl}').result().to_dataframe()
      .drop(columns=[district_type]).rename(columns={'geoid':district_type})
     )
geo = gpd.GeoSeries.from_wkt(df['polygon'], crs='EPSG:4326').simplify(0.05).buffer(0)
gdf = gpd.GeoDataFrame(df, geometry=geo)
gdf['color'] = gdf['color'].astype(str)
gdf['locations'] = gdf.index

def to_csv():
    df.drop(columns=[district_type, 'polygon', 'point']).to_csv(root_path / 'recomb_data.csv')

def make_images(animate=False, num_steps=1000000):
    num_steps = min(num_steps, df['plan'].max())
    if animate:
        opts['height'] = 1.13 * opts['width']
        X = gdf.query(f'plan <= {num_steps}')
        fig = px.choropleth_mapbox(X, geojson=X.geometry, **opts, animation_frame='plan', animation_group=X[district_type])
        fig.update_geos(fitbounds="locations", visible=False)
        fig.update(layout_showlegend=False)
        f = image_path / f'recomb_{district_type}_{level}.html'
        f.parent.mkdir(parents=True, exist_ok=True)
        fig.write_html(f, auto_play=True)
    else:
        opts['height'] = 0.97 * opts['width']
        for k in range(num_steps+1):
            print(k)
            X = gdf.query(f'plan == {k}')
            fig = px.choropleth_mapbox(X, geojson=X.geometry, **opts)
            fig.update_geos(fitbounds="locations", visible=False)
            fig.update(layout_showlegend=False)
            dig = len(str(num_steps))
            for ext in ['png',
                        'svg',
                        'eps',
                        'html']:
                f = image_path / f'{ext}/recomb_{district_type}_{level}_{str(k).rjust(dig,"0")}.{ext}'
                f.parent.mkdir(parents=True, exist_ok=True)
                if ext == 'html':
                    fig.write_html(f)
                else:
                    fig.write_image(f)
    return fig

opts = {
    'locations' : 'locations',
    'color' : "color",
    'color_discrete_sequence' : px.colors.qualitative.T10,
    'mapbox_style' : 'carto-positron',
    'opacity' : .4,
    'center' : {'lon': -100.05, 'lat': 31.0},
    'hover_data' : {'color':False,
                    'locations':False,
                    'cd': True,
                    'plan': True,
                    'total_pop': True,
                    'polsby_popper': True
                   }
}

# opts['zoom'] = 3.0; opts['width'] = 310
opts['zoom'] = 4.0; opts['width'] = 460
# opts['zoom'] = 5.0; opts['width'] = 760

fig = make_images(animate=False)#, num_steps=1)
fig.show()
fig = make_images(animate=True, num_steps=20)
fig.show()

KeyboardInterrupt: 

In [ ]:
I'm making animations where geospatial polygons get re-colored.  The code works great, except the file size is too large for the use case.  Any suggestions to shrink the file size without losing functionality.

opts = {
    'locations' : 'locations',
    'color' : 'color',
    'color_discrete_sequence' : px.colors.qualitative.T10,
    'mapbox_style' : 'carto-positron',
    'opacity' : 0.4,
    'center' : {'lon': -100.05, 'lat': 31.0},
    'hover_data' : {'color':False,
                    'locations':False,
                    'cd': True,
                    'plan': True,
                    'total_pop': True,
                    'polsby_popper': True
                   }
}
fig = px.choropleth_mapbox(X, geojson=X.geometry, **opts)
fig.update_geos(fitbounds='locations', visible=False)
fig.write_html(file, auto_play=True)